## Fairness metrics to evaluate the model

In [1]:
# Statistical Parity Difference
def spd(sensitive_attribute, dataset, predicted_labels, majority_class, minority_class):
    """
    Calculate the Statistical Parity Difference (SPD) between majority and minority classes based on predicted labels.

    Parameters:
    - sensitive_attribute (str): Name of the column representing the sensitive attribute.
    - dataset (pd.DataFrame): The dataset containing the sensitive attribute and true outcome variable.
    - predicted_labels (pd.Series): Predicted labels for the outcome variable.
    - majority_class: Value representing the majority class in the sensitive attribute.
    - minority_class: Value representing the minority class in the sensitive attribute.

    Returns:
    - spd (float): Statistical Parity Difference between majority and minority classes.
    """
    predicted_labels = pd.to_numeric(predicted_labels)
    predicted_labels_series = pd.Series(predicted_labels, index=dataset.index)
    majority = dataset[dataset[sensitive_attribute] == majority_class]
    minority = dataset[dataset[sensitive_attribute] == minority_class]

    p_majority = predicted_labels_series[majority.index].mean()
    p_minority = predicted_labels_series[minority.index].mean()

    spd_val =  p_minority - p_majority
    return spd_val

In [2]:
# Disparate Impact
def di(sensitive_attribute, dataset, predicted_labels, majority_class, minority_class):
    """
    Calculate the Disparate Impact (DI) between majority and minority classes based on predicted labels.

    Parameters:
    - sensitive_attribute (str): Name of the column representing the sensitive attribute.
    - dataset (pd.DataFrame): The dataset containing the sensitive attribute and true outcome variable.
    - predicted_labels (pd.Series): Predicted labels for the outcome variable.
    - majority_class: Value representing the majority class in the sensitive attribute.
    - minority_class: Value representing the minority class in the sensitive attribute.

    Returns:
    - di (float): Disparate Impact between majority and minority classes.
    """
    predicted_labels = pd.to_numeric(predicted_labels)
    predicted_labels_series = pd.Series(predicted_labels, index=dataset.index)
    majority = dataset[dataset[sensitive_attribute] == majority_class]
    minority = dataset[dataset[sensitive_attribute] == minority_class]

    p_majority = predicted_labels_series[majority.index].mean()
    p_minority = predicted_labels_series[minority.index].mean()

    di_val = p_minority / p_majority
    return di_val

In [3]:
# spd for current dataset
"""
Range: The range of SPD is [-1, 1]. 
A value of -1 indicates that all favorable outcomes are allocated to the majority group, 
whereas a value of 1 indicates that all favorable outcomes are allocated to the minority group. 
Perfect Fairness: A value of 0 indicates perfect fairness, 
meaning the probability of receiving a favorable outcome is equal for both the majority and minority groups.
"""

spd("Gender", df_prediction, df_prediction["Interview prediction"], "M", "F")

NameError: name 'df_prediction' is not defined

In [ ]:
# di for current dataset
"""
Range: DI is a ratio, so its range is [0, ∞). 
A value of 0 indicates extreme bias against the minority group, 
and a very high value indicates extreme bias against the majority group. 
Perfect Fairness: A DI of 1 (or close to 1) represents perfect fairness, 
suggesting that the probability of receiving a favorable outcome are equal for both groups.
"""

di("Gender", df_prediction, df_prediction["Interview prediction"], "M", "F")